In [170]:
# data from CDC: https://data.cdc.gov/Case-Surveillance/United-States-COVID-19-Cases-and-Deaths-by-State-o/9mfq-cb36

import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt

from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, CustomJS, Dropdown, CategoricalColorMapper
from bokeh.layouts import column
from bokeh.io import show
from bokeh.palettes import viridis

# Make the graphs a bit prettier, and bigger
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 5)
sns.set(font_scale=1.5, style="whitegrid")

df = pd.read_csv("C:\\Users\\arerlend\\Downloads\\United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv")

df['submission_date']= pd.to_datetime(df['submission_date'])
df.drop(df[df['new_case'] < 0].index, inplace=True)

y_var = "tot_cases"

# print(df.head())
# fig, ax = plt.subplots(figsize=(12, 12))

# # Add x-axis and y-axis
# ax.plot(df_wa['submission_date'],
#         df_wa[y_var],
#         color='purple')
# # Add x-axis and y-axis
# ax.plot(df_ny['submission_date'],
#         df_ny[y_var],
#         color='blue')

# # Set title and labels for axes
# ax.set(xlabel="Date",
#        ylabel=y_var,
#        title=y_var)

# plt.show()

p = figure(title=y_var, x_axis_label='submission date', y_axis_label="total cases", x_axis_type='datetime')

source = ColumnDataSource(df)
render = ColumnDataSource(data=dict(x=df["submission_date"], y=df[y_var]))

menu = df.state.unique().tolist()
legend_label="total cases"
# add a line renderer with legend and line thickness
p.line(x='x', y='y', source=render, line_width=2)

print(df[y_var].describe())

#plot and the menu is linked with each other by this callback function
callback = CustomJS(args=dict(source=source, render=render), code="""
    var data = source.data
    var new_x = []
    var new_y = []
    for (var i = 0; i < data.state.length; i++) {
        if (data.state[i] == this.item) {
            new_x.push(data.submission_date[i])
            new_y.push(data.tot_cases[i])
        }
    }
    
    var f = cb_obj.value
    render.data.x = new_x
    render.data.y = new_y
    console.log(source.data.tot_cases.length)

    render.change.emit()
""")

dropdown = Dropdown(label="Dropdown button", button_type="warning", menu=menu)
dropdown.js_on_event("menu_item_click", callback)


# show the result
layout = column(p, dropdown)
show(layout)

count     16600.000000
mean      48147.461145
std      102252.296275
min           0.000000
25%          19.000000
50%        7105.000000
75%       52949.500000
max      898029.000000
Name: tot_cases, dtype: float64
